We will load the input data as dataframe:

In [29]:
import os
from sqlalchemy import create_engine
from tqdm import tqdm
import psycopg2
import json
import pandas as pd
import numpy as np

with open('../credentials.json') as f:
    data = json.load(f)
    psql_config = {
    'dbname': data['db_name'],
    'user': data['db_user'],
    'password': data['db_pwd'],
    'host': data['db_host'],
    'port': 5432
}


def get_psql_connection():
    try:
        conn = psycopg2.connect(**psql_config)
    except Exception as e:
        print("Error connecting to the database:", e)
    return conn

In [2]:
engine = create_engine(f'postgresql://{psql_config["user"]}:{psql_config["password"]}@{psql_config["host"]}/{psql_config["dbname"]}')

df = pd.read_sql("SELECT * FROM m024.citi_bike_data LIMIT 1000;", engine)

df.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,processed
0,314,2018-04-15 12:17:02.635,2018-04-15 12:22:17.257,499,Broadway & W 60 St,40.769155,-73.981918,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,30250,Subscriber,1981,1,False
1,329,2018-04-15 12:26:40.715,2018-04-15 12:32:10.258,499,Broadway & W 60 St,40.769155,-73.981918,3167,Amsterdam Ave & W 73 St,40.779668,-73.980930,30491,Subscriber,1967,1,False
2,124,2018-04-15 12:38:16.050,2018-04-15 12:40:20.975,499,Broadway & W 60 St,40.769155,-73.981918,423,W 54 St & 9 Ave,40.765849,-73.986905,15480,Subscriber,1978,1,False
3,248,2018-04-15 12:43:27.859,2018-04-15 12:47:36.266,499,Broadway & W 60 St,40.769155,-73.981918,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,30922,Subscriber,1988,1,False
4,1121,2018-04-15 12:47:25.606,2018-04-15 13:06:07.452,499,Broadway & W 60 St,40.769155,-73.981918,3163,Central Park West & W 68 St,40.773407,-73.977825,32707,Customer,1990,1,False


Now that we have the data, we will start processing it in bunch.
For each chunk, we will apply some validation and transform the source data into fact and dimension data

In [31]:
from math import radians, cos, sin, sqrt, atan2
from psycopg2.extras import execute_values
BATCH_SIZE = 10

# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# Extract Data
def extract_data(offset):
    query = f"""
    SELECT * FROM m024.citi_bike_data 
    WHERE processed = FALSE 
    ORDER BY ctid
    LIMIT {BATCH_SIZE} OFFSET {offset}
    """
    return pd.read_sql(query, engine)

# Load data into dimension tables
def load_dimension_data(df, table_name, cols, db_cols):
    df = df[cols].astype('str').drop_duplicates()
    try:
        with get_psql_connection() as conn:
            with conn.cursor() as cur:
                statement = f"""
                INSERT INTO m024.p_{table_name} ({', '.join(db_cols)})
                VALUES %s 
                ON CONFLICT DO NOTHING;"""
                data_tuples = [tuple(row) for row in df.to_numpy()]
                    
                # Execute batch insert
                execute_values(cur, statement, data_tuples)
                
                # Commit changes
                conn.commit() 
                print(f"Data inserted/updated successfully in {table_name}")
    except Exception as e:
        print(f"Error in inserting/updating data for table {table_name}- {e}")

# Load dimension tables and return mapping IDs
def get_dimension_id(df, table_name, lookup_col, db_lookup_col,db_return_col):
    lookup_values = df[lookup_col].drop_duplicates().tolist()
    query = f"SELECT {db_lookup_col}, {db_return_col} FROM m024.p_{table_name} WHERE {db_lookup_col} IN %s"
    mapping = pd.read_sql(query, engine, params=(tuple(lookup_values),))
    return dict(zip(mapping[db_lookup_col], mapping[db_return_col]))

def get_time_dimension_id(df, table_name, lookup_col, db_lookup_cols, db_return_col):
    lookup_values = df[lookup_col].drop_duplicates().tolist()

    # Build query based on date components to avoid precision issues with timestamps
    query = f"""
    SELECT {', '.join(db_lookup_cols)}, {db_return_col}
    FROM m024.p_{table_name}
    WHERE ({', '.join(db_lookup_cols)}) IN %s
    """
    lookup_tuples = [
        (row['year'], row['month'], row['day'], row['hour']) for _, row in df.iterrows()
    ]
    
    mapping = pd.read_sql(query, engine, params=(tuple(lookup_tuples),))
    return dict(zip(mapping[db_lookup_cols], mapping[db_return_col]))


In [ ]:
# Transform Data
def transform_data(df):

    # transform gender and load the dimension
    df.gender = df.gender.map(dict(zip([1, 2],['Male','Female']))).fillna('Unknown')
    load_dimension_data(df, 'gender_dimension', ['gender'],['gender_type'])

    # transform user type and load the dimension
    df.usertype = df.usertype.apply(lambda x: x if x in ['Subscriber', 'Customer'] else 'Unknown')
    load_dimension_data(df, 'user_type_dimension', ['usertype'],['user_type'])

    # transform birth year and load the dimension
    df.birth_year = df.birth_year.apply(lambda x: x if x > 1940 and x <2013 else 0).astype(int) # Assuming you need to be atleast 5 to ride the bike
    load_dimension_data(df, 'user_birthyear_dimension', ['birth_year'],['user_birthyear'])

    # Clean station names
    df['start_station_name'] = df['start_station_name'].str.strip().fillna('Unknown')
    df['end_station_name'] = df['end_station_name'].str.strip().fillna('Unknown')

    # Validate Latitude and Longitude
    df['start_station_latitude'] = df['start_station_latitude'].apply(
        lambda x: x if -90 <= x <= 90 else None
    )
    df['start_station_longitude'] = df['start_station_longitude'].apply(
        lambda x: x if -180 <= x <= 180 else None
    )
    df['end_station_latitude'] = df['end_station_latitude'].apply(
        lambda x: x if -90 <= x <= 90 else None
    )
    df['end_station_longitude'] = df['end_station_longitude'].apply(
        lambda x: x if -180 <= x <= 180 else None
    )

    # For missing latitude/longitude values
    df['start_station_latitude'].fillna('Unknown', inplace=True)
    df['start_station_longitude'].fillna('Unknown', inplace=True)
    df['end_station_latitude'].fillna('Unknown', inplace=True)
    df['end_station_longitude'].fillna('Unknown', inplace=True)
    load_dimension_data(df, 'station_dimension',
                         ['start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude'],
                           ['station_key', 'station_name', 'latitude', 'longitude'])
    load_dimension_data(df, 'station_dimension',
                         ['end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude'],
                           ['station_key', 'station_name', 'latitude', 'longitude'])
    
    # Fill missing values
    df['starttime'].fillna('1.1.1900', inplace=True)
    df['stoptime'].fillna('1.1.1900', inplace=True)

    # Convert to datetime format
    df['starttime_dt'] = pd.to_datetime(df['starttime'])
    df['stoptime_dt'] = pd.to_datetime(df['stoptime'])

    # Extract fields for start time
    df['start_date'] = df['starttime_dt'].dt.date
    df['start_year'] = df['starttime_dt'].dt.year
    df['start_month'] = df['starttime_dt'].dt.month
    df['start_day'] = df['starttime_dt'].dt.day
    df['start_hour'] = df['starttime_dt'].dt.hour

    # Extract fields for stop time
    df['stop_date'] = df['stoptime_dt'].dt.date
    df['stop_year'] = df['stoptime_dt'].dt.year
    df['stop_month'] = df['stoptime_dt'].dt.month
    df['stop_day'] = df['stoptime_dt'].dt.day
    df['stop_hour'] = df['stoptime_dt'].dt.hour

    load_dimension_data(df, 'time_dimension',
                         ['starttime', 'start_date', 'start_year', 'start_month', 'start_day', 'start_hour'],
                         ['time', 'date', 'year', 'month', 'day', 'hour'])
    
    load_dimension_data(df, 'time_dimension',
                         ['stoptime', 'stop_date', 'stop_year', 'stop_month', 'stop_day', 'stop_hour'],
                         ['time', 'date', 'year', 'month', 'day', 'hour'])
    
    load_dimension_data(df, 'bike_dimension', ['bikeid'], ['bike_id'])

    df['starttime_dt'] = pd.to_datetime(df['starttime'])
    df['stoptime_dt'] = pd.to_datetime(df['stoptime'])

    # Extract fields for start time
    df['start_date'] = df['starttime_dt'].dt.date
    df['start_year'] = df['starttime_dt'].dt.year
    df['start_month'] = df['starttime_dt'].dt.month
    df['start_day'] = df['starttime_dt'].dt.day
    df['start_hour'] = df['starttime_dt'].dt.hour

    # Extract fields for stop time
    df['stop_date'] = df['stoptime_dt'].dt.date
    df['stop_year'] = df['stoptime_dt'].dt.year
    df['stop_month'] = df['stoptime_dt'].dt.month
    df['stop_day'] = df['stoptime_dt'].dt.day
    df['stop_hour'] = df['stoptime_dt'].dt.hour

    load_dimension_data(df, 'time_dimension',
                         ['starttime', 'start_date', 'start_year', 'start_month', 'start_day', 'start_hour'],
                         ['time', 'date', 'year', 'month', 'day', 'hour'])

    load_dimension_data(df, 'time_dimension',
                         ['stoptime', 'stop_date', 'stop_year', 'stop_month', 'stop_day', 'stop_hour'],
                         ['time', 'date', 'year', 'month', 'day', 'hour'])

    # Calculate trip distance
    df['distance'] = df.apply(lambda row: haversine(
        row['start_station_latitude'], row['start_station_longitude'], 
        row['end_station_latitude'], row['end_station_longitude']
    ), axis=1)

    # Get dimension table mappings
    station_map = get_dimension_id(df, 'station_dimension', 'start_station_id', 'station_key','station_id')
    end_station_map = get_dimension_id(df, 'station_dimension', 'end_station_id', 'station_key','station_id')
    start_time_map = get_dimension_id(df, 'time_dimension', 'starttime', 'time', 'time_id')
    stop_time_map = get_dimension_id(df, 'time_dimension', 'stoptime', 'time', 'time_id')
    user_type_map = get_dimension_id(df, 'user_type_dimension', 'usertype', 'user_type','user_type_id')
    gender_map = get_dimension_id(df, 'gender_dimension', 'gender', 'gender_type','gender_id')
    birth_year_map = get_dimension_id(df, 'user_birthyear_dimension', 'birth_year', 'user_birthyear', 'user_birthyear_id')

    # Map dimension table IDs
    df['start_time_id'] = df['starttime'].map(start_time_map)
    df['end_time_id'] = df['stoptime'].map(stop_time_map)
    df['start_station_id'] = df['start_station_id'].map(station_map)
    df['end_station_id'] = df['end_station_id'].map(end_station_map)
    df['bike_id'] = df['bikeid']
    df['user_type_id'] = df['usertype'].map(user_type_map)
    df['gender_type_id'] = df['gender'].map(gender_map)
    df['user_birthyear_id'] = df['birth_year'].map(birth_year_map)
    df['duration'] = df['tripduration']

    return df[['duration', 'distance', 'start_time_id', 'end_time_id', 'start_station_id', 'end_station_id', 'bike_id', 'user_type_id', 'gender_type_id', 'user_birthyear_id']]

# Update processed records
def update_processed(df, limit=BATCH_SIZE, offset=0):
    query = f"""
    WITH cte AS (
        SELECT ctid
        FROM m024.citi_bike_data
        WHERE processed = FALSE
        ORDER BY ctid
        LIMIT {BATCH_SIZE} OFFSET {offset}
    )
    UPDATE m024.citi_bike_data
    SET processed = TRUE
    WHERE ctid IN (SELECT ctid FROM cte);
    """
    
    try:
        with get_psql_connection() as conn:
            with conn.cursor() as cur:
                # Execute the update statement
                cur.execute(query)
                conn.commit()  # Commit the changes
                print(f"Successfully updated the processed flag for {limit} rows (Offset: {offset}).")
    except Exception as e:
        print(f"Error in updating processed flag: {e}")

# Process batch
def process_batch(offset):
    df = extract_data(offset)
    if not df.empty:
        df_transformed = transform_data(df)
        load_dimension_data(df_transformed, 'trip_fact', df_transformed.columns, df_transformed.columns)
        #return 0
        update_processed(df)
        print(f"Processed batch {offset}-{offset+BATCH_SIZE}")


process_batch(0)


Data inserted/updated successfully in gender_dimension
Data inserted/updated successfully in user_type_dimension
Data inserted/updated successfully in user_birthyear_dimension


C:\Users\Besitzer\AppData\Local\Temp\ipykernel_26052\1233814397.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['start_station_latitude'].fillna('Unknown', inplace=True)
C:\Users\Besitzer\AppData\Local\Temp\ipykernel_26052\1233814397.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

Data inserted/updated successfully in station_dimension
Data inserted/updated successfully in station_dimension


C:\Users\Besitzer\AppData\Local\Temp\ipykernel_26052\1233814397.py:47: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['starttime'].fillna('1.1.1900', inplace=True)
C:\Users\Besitzer\AppData\Local\Temp\ipykernel_26052\1233814397.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


Data inserted/updated successfully in time_dimension
Data inserted/updated successfully in time_dimension
Data inserted/updated successfully in bike_dimension
Data inserted/updated successfully in time_dimension
Data inserted/updated successfully in time_dimension
Data inserted/updated successfully in trip_fact
Successfully updated the processed flag for 100 rows (Offset: 0).
Processed batch 0-10
